In [ ]:
# Libraries to access external data
# PyVO provides generic access to VO protocols (ex: TAP)
import pyvo
# astroquery modules are specific to astronomy services, there are not necessarily VO,
# but more and more do switch to using VO protocols behind the scenes these days
# here we import the MOCServer and Vizier modules
from astroquery.mocserver import MOCServer

# The mocpy library is an astropy-affiliated library
# It allows to manipulate MOCs
from mocpy import STMOC, MOC, TimeMOC

# General astropy utility methods and classes
from astropy.coordinates import Longitude, Latitude, Angle, SkyCoord
from astropy.io.votable import parse_single_table
from astropy.time import Time, TimeDelta
import astropy.units as u

# For visualization
from ipyaladin import Aladin


# Accessing data through space and time criteria

## This notebook focuses on the use of the Multi-Order Coverage (MOC) and Space-Time MOC (STMOC) for querying and analyzing astronomical data. 

### Key words:

**Hierarchical Progressive Survey (HiPS)**: HiPS describes a generic method for packaging, storing, querying, and describing astronomical data. It stores data in a folder structure that mimics the [HEALPix tesselation of the sky](https://healpix.jpl.nasa.gov/pdf/intro.pdf).

**Multi-Order Coverage (MOC)**: MOC is a data structure used to describe arbitrary regions of the sky with variable resolution. It is particularly useful for handling large-scale astronomical surveys and for efficiently querying regions of interest.

**Space-Time MOC (STMOC)**: STMOC extends the concept of MOC to include the temporal dimension, allowing for the representation of regions of the sky that change over time. This is useful for time-domain astronomy, where the visibility of sources varies.

### Scientific Motivation:

Gamma-Ray Bursts (GRBs) are among the most energetic and mysterious phenomena in the universe. These brief but intense flashes of gamma rays are believed to originate from cataclysmic events such as the collapse of massive stars or the merger of neutron stars. Observing GRBs across multiple wavelengths and with different telescopes provides crucial insights into their nature and origins. In this study, we focus on GRBs observed by the Fermi Gamma-ray Space Telescope and the XMM-Newton X-ray Observatory, and demonstrate how the use of Space-Time Multi-Order Coverage (STMOC) can help identify and analyze these fascinating events.

## The HEALPix grid and its link to HiPS surveys

HiPS is a way of storing data sets (it can be catalogs, or images for example) that follows the HEALPix tesselation of the sky. This is particularly optimized for visualization.

![HiPS_and_the_HEALPix_grid](HiPS_and_the_HEALPix_grid.png)

Figure 1: Standards to represent the sky, or parts of the sky. (a) The HEALPix grid (b) The HiPS standard (c) Visualization in ipyaladin (d), and a MOC.

### The HEALPix grid

With the HEALPix grid, the sphere is mapped by equal-area tiles. In **Figure 1a**, we show an orthographic representation of the 12 tiles at order zero and of the 768 tiles at order 3. This is extracted from [Gorski *et.al.*](https://iopscience.iop.org/article/10.1086/427976/pdf).

`mocpy` is a python library that allows to manipulate MOCs. MOCs are made of HEALPix cells (see **Figure 1d** and the next parts where we'll look at MOCs in more details), so mocpy has a few methods that allow to get information about the HEALPix tesselation.

In [ ]:
# we can check the number of cells at order 0 in the HEALPix tesselation
MOC.n_cells(0)

In [ ]:
# or at an other order
MOC.n_cells(3)

In [ ]:
# we could also have calculated the order 3 by knowing that
# each cell is subdivised in four parts at the next order
n_cells_order3 = MOC.n_cells(0) * 4**3
n_cells_order3

Let's visualize the HEALPix grid in the ipyaladin widget, see **Figure 1c** to see where to find the parameter. If you zoom in and out and move like crazy (or if you have a slower internet connection), you'll see the images that appear as they are downloaded by ipyaladin. They have the shape of HEALPix cells. The more you zoom in, the more precise (meaning higher resolution) the printed image will be.

In [ ]:
aladin = Aladin()
aladin

We can duplicate this view by right clicking on the cell and selecting `Create new view for cell output`. The view can then be dragged anywhere you prefer. This allows to always have an eye on the widget while exploring the data.
In the `Output View`, it is also quite nice to use the `fullscreen` button, the one in the top right corner of the widget.

If you zoom in **a lot**, you will see that the pixels correspond to the tiles of the HEALPix grid at order 18 for the DSS survey that is displayed by default in `ipyaladin`. You can find out which resolution it corresponds to with an other of `mocpy`'s methods:

In [ ]:
MOC.order_to_spatial_resolution(18)

Which we can also convert to arcseconds (or any other angle unit, try it out!)

In [ ]:
MOC.order_to_spatial_resolution(18).to("arcsec")

### The HiPS standard

The images displayed by Aladin follow the HiPS standard, in which a folder corresponds to a HEALPix order, see **figure1b**. In figure 1b, the example images are from a SDO/AIA observation of the Sun. At all orders, the images have the same number of pixels (here 512*512), but they represent a different sky fraction. On the figure, we see that the pixel 0 at order 3 contains the pixel 0 and 1 of the more precise order.

### Finding a HiPS survey

More and more astronomical data centers start to offer HiPS surveys for sky visualizations.

One way of finding HiPS surveys is to use `ipyaladin`'s interface, see **Figure2**, and follow the steps to get the two MeerKAT HiPS.

![finding_HiPS](finding_hips_with_ipyaladin.png)
**Figure 2**: Finding a HiPS with `ipyaladin`'s interface.

Nothing appears? Let's move the target to the galactic center and zoom in a bit:

In [ ]:
aladin.target = "Galactic Center"
aladin.fov = Angle(5, unit="deg") # the Angle object is from astropy

And you should see the MeerKat survey! 

Note that `fov`, the field of view argument, fixes the angle on the x axis.

#### Finding HiPS programmatically

The `Browse HiPS` button calls one of the less known services of the CDS, the MOCServer. Its name is a bit misleading, as it does not only hosts MOCs, but also HiPSs. The search bar in `ipyaladin` only works if you already know more or less the name of the survey. For more control on search parameters, we will use the module `mocserver` from the `astroquery` library that give a direct access the the [MOCServer](https://astroquery.readthedocs.io/en/latest/mocserver/mocserver.html#module-astroquery.mocserver).

Here we query the MOC server to find HiPS datasets related to XMM observations.

#### XMM-Newton

GRBs can be studied in conjunction with other instruments spanning wide range of wavelengths in order to understand the various properties of the GRB. The bursts of gamma-rays produced by the GRBs collide with each other and also the interstellar medium and produce what we call the "after glow". The "after glow" contains electromagnetic radiations ranging from X-rays to radio. Let's explore the GRBs also detected in X-rays. As shown above, we can obtain HiPs for the XMM-Newton Telescope. XMM-Newton is a powerful telescope that observes the X-ray Universe.


In [ ]:
help(MOCServer)

In [ ]:
xmm_hips = MOCServer.find_datasets(meta_data="obs_description=*xmm*&&hips_*=*", casesensitive=False)
xmm_hips

In [ ]:
xmm_hips["ID"]

We get four HiPS, one is a color composition: `xcatdb/P/XMM/PN/color`

#### Setting surveys in ipyaladin from python

In [ ]:
aladin.overlay_survey="xcatdb/P/XMM/PN/color"

In [ ]:
aladin.overlay_survey_opacity = 0.8

That's great, we now have superposed three surveys in `ipyaladin`.
HiPS surveys are built out of mosaic images. We see from the XMM example that they do not necessarily cover the whole sky, but can also be constructed for partial coverages.

We will not build a HiPS catalog now, but if one day you'd like to build and publish your own, you can look at [this very nice HiPS building tutorial](https://aladin.cds.unistra.fr/tutorials/OV-France-2024/)

Let's now see how we can retrieve GRB events that could have been observed by these telescopes.

## Interacting with databases programmatically

### Using the Table Access Protocol (TAP) with pyVO

TAP is a VO standard that allows to interrogate tables that have relations with each other with the Astronomical Data Query Language (ADQL).

A database that offers TAP access always have:
- an url that allows to instantiate a TAP service
- a description of its content that is always in the same tables with standardized names. Here we will use **tables**, and **columns**.

## Different python libraries

There are two main ways to interact with astronomical databases from python.
- with `pyVO`, the python module that offers Virtual Observatory protocols,
- with `astroquery` modules. If there is one that exists for your database, then it is often really nice to use. But **it is less general than pyVO that works for every VO database**. We used this second approach for the MOCServer, `astroquery.mocserver` was an astroquery module specially written for the MOCServer.

Here, we will focus on the `pyVO` approach, to stick to VO protocols. But most [CDS's python tutorials](https://github.com/cds-astro/tutorials/notebooks) will illustrate the modules `astroquery.vizier`, `astroquery.xmatch`and `astroquery.simbad`, if you are interested.


#### SIMBAD

![tap_simbad](tap_simbad.png)
Figure 3: (a) Finding the SIMBAD TAP URL, (b) The SIMBAD tables `basic` and `otypedef` have a common column: `otype`, (c) Once a TAP result is added to `ipyaladin`, one can click on the symbols or select subsets to inspect the results further.

Let's try to find GRBs with SIMBAD's TAP access.
First, we need SIMBAD's TAP URL. It can be found on [SIMBAD's webpage](http://simbad.cds.unistra.fr/simbad/), see **Figure 3a**.

In [ ]:
tap_simbad = pyvo.dal.TAPService("https://simbad.cds.unistra.fr/simbad/sim-tap/")

The queries written in ADQL should be written in a string and called with the `search` method. Let's look at the tables available through TAP. `*` means that we want to get everything, and `FROM tables` restricts is to the table named `tables`.

In [ ]:
tap_simbad.search("SELECT * FROM tables").to_table()

This ADQL query (`SELECT * FROM tables`) will work in every TAP service, since `tables` is one of the mandatory tables.

Here, we find that SIMBAD is actually made of 35 tables. Some of them have a name that start by `TAP_SCHEMA`. These are the mandatory tables that describe the TAP service.

An other query that will always work is to get the columns in one of the tables of a TAP service, since `columns` is also one of the mandatory tables. Let's retrieve the descriptions of the columns of the `basic` table.

In [ ]:
tap_simbad.search("SELECT * FROM TAP_SCHEMA.columns WHERE table_name = 'basic'").to_table()

The tables that don't start with `TAP_SCHEMA` are specific to each database.

Let's go back to our question. How do we find which GRBs are in SIMBAD?
We'll use a criteria on the `otype`, which is `gB` for grbs in SIMBAD. We could have found that on the [object types page of SIMBAD](http://simbad.cds.unistra.fr/guide/otypes.htx). The next commented cells show how to do this from python in ADQL queries. This is a nice example about using other tables than `basic`. If you're a bit faster than the group, you're welcome to uncomment and read through the process.

#### Optional

First, we'll need to find how GRB's are called. To find which tables of SIMBAD offers a description of the object types, let's add a criteria to our first query:

In [ ]:
#tap_simbad.search("SELECT * FROM tables WHERE description LIKE '%type%'").to_table()

In ADQL, `%` means 'any character'. We learn that the definition of the object types in SIMBAD is in the table `otypedef`. Let's look at the first 5 records of the table to see what it looks like.

In [ ]:
#tap_simbad.search("SELECT TOP 5 * FROM otypedef").to_table()

We can now build an ADQL query to find the code `otype` (the first column) for Gamma Ray Bursts. Since only the columns `otype`, `description`, and `path` look interesting to us, let's do a query without `*` for the first time:

In [ ]:
#tap_simbad.search("SELECT otype, description, path FROM otypedef WHERE description LIKE '%Gamma%'").to_table()

We got it. The code is `gB`. 

This information was extracted from one of SIMBAD's tables. We will use an other one in the next query. If you're interested in learning more about SIMBAD's structure, and how to build more complex ADQL queries for it, you can have a look at [SIMBAD's ADQL documentation](https://cds.unistra.fr/help/documentation/simbad-more/adql-simbad/), or the [documentation of the `astroquery.simbad` module](https://astroquery.readthedocs.io/en/latest/simbad/simbad.html#query-tap) that has a `query_tap` direct access.

#### End of optional

Let's now do a request on the `basic` table that contains all the basic information about the objects. `basic` has a lot on columns. You can inspect them with an ADQL query (`SELECT * FROM columns WHERE table_name = 'basic'`) but we already selected the coordinates (`ra`, `dec`), the object type (`otype`) and the number of articles that mention the GRB (`nbref`):

In [ ]:
grbs_simbad = tap_simbad.search("""SELECT main_id, ra, dec, otype, nbref FROM basic
WHERE otype = 'gB'""").to_table()

# or without internet access you can uncomment the next line
# grbs_simbad = parse_single_table("grbs_simbad.vot").to_table()

grbs_simbad

There are currently 12449 GRBs in SIMBAD. There could be more when you'll execute the notebook. Some of them have position estimates, some don't. This is because SIMBAD is also a bibliographical database, it does not only map objects onto the sky, it also maps objects mentioned in papers. 

For example here, let's look at the GRB with the most citations (`nbref`)

In [ ]:
grbs_simbad.sort("nbref", reverse=True)
grbs_simbad

It looks like the "most famous" GRB is `GRB 990123`.

And indeed, GRB 990123 is a gamma-ray burst which was detected on January 23, 1999. It was the first GRB for which a simultaneous optical flash was detected. 
In the dark hours of the morning of January 23, 1999, the Compton satellite recorded a gamma-ray burst that lasted for about a minute and a half. There was a peak of gamma and X-ray emission 25 seconds after the event was first detected, followed by a somewhat smaller peak 40 seconds after the beginning of the event. The emission then fizzled out in a series of small peaks over the next 50 seconds, and eight minutes after the event had faded to a hundredth of its maximum brightness. The burst was so strong that it ranked in the top 2% of all bursts detected.
Image by the Keck telescope, and the 2.6 meter Nordic Optical Telescope in the Canary Islands, revealed absorption lines with a redshift of 1.6, implying a distance of 9 billion light-years.

The second in the list, GRB 970508, is historically important as the second GRB (after GRB 970228) with a detected afterglow at other wavelengths, the first to have a direct redshift measurement of the afterglow, and the first to be detected at radio wavelengths.
Astronomer Mark Metzger determined that GRB 970508 occurred at least 6 billion light years from Earth; this was the first measurement of the distance to a gamma-ray burst. Until this burst, astronomers had not reached a consensus regarding how far away GRBs occur from Earth. Some supported the idea that GRBs occur within the Milky Way, but are visibly faint because they are not highly energetic. Others concluded that GRBs occur in other galaxies at cosmological distances and are extremely energetic. Although the possibility of multiple types of GRBs meant that the two theories were not mutually exclusive, the distance measurement unequivocally placed the source of the GRB outside the Milky Way, effectively ending the debate.
GRB 970508 was also the first burst with an observed radio frequency afterglow. By analyzing the fluctuating strength of the radio signals, astronomer Dale Frail calculated that the source of the radio waves had expanded almost at the speed of light. This provided strong evidence that GRBs are relativistically expanding explosions.

### Adding tables to the ipyaladin view

Let's add the GRBs from SIMBAD with an estimated position to the `ipyaladin` widget.

In [ ]:
aladin.add_table(grbs_simbad, color="pink", shape="cross", name="simbad_grbs")

We immediately see that some fall within the XMM observations. This is nice, but GRBs have a short lifetime span and SIMBAD only has space information, not time. 

We will need to go to more specific data sources. To do so, let's find other TAP services that could have GRBs-specific information.

#### But where do we find TAP links?? PyVO

For this, we will use one extra-powerful tool: the Virtual Observatory registry. This is where any VO service is... registered! It is accessible through [web access](http://dc.g-vo.org/wirr/q/ui/fixed). It is also what TOPCat uses behind the scenes when you're looking for a resource. Here, we will access it thanks to `PyVO`.

The pyVO registry module works with constraints that you can define and then apply to a query. The most common ones are:

- `FreeText`
- `Waveband`
- `ServiceType`
- `Author`
- `UCD`

But you can find a lot of other ones on the [registry module's documentation](https://pyvo.readthedocs.io/en/latest/registry/index.html).

#### How do we specifically look for time information?? The UCD standard

UCD means Unified Content Descriptor and is a description of the content of the columns of a VO resource.

You can get the UCD corresponding to a word on the [UCD Builder page](https://cds.unistra.fr//UCD/cgi-bin/descr2ucd). Here if we enter `time` we get the suggestion to use `time.epoch` as an UCD. Let's try that in the registry. A more complete documentation about UCD can be found on the [VO website](https://ivoa.net/documents/REC/UCD/UCD-20050812.html#ToC9).

In [ ]:
# we first define text, servicetype and UCD constraints
text_constraint = pyvo.registry.Freetext("Gamma", "Ray", "Burst")
service_type_constraint = pyvo.registry.Servicetype("tap").include_auxiliary_services()
ucd_constraint_time = pyvo.registry.UCD("time.epoch")


tap_services_GRBs = pyvo.registry.search(text_constraint, service_type_constraint, ucd_constraint_time)
tap_services_GRBs

In [ ]:
tap_services_GRBs.fieldnames

In [ ]:
tap_services_GRBs.to_table()[["res_title", "short_name"]]

We find more than 160 services that correspond to our criteria. We can inspect them further by reading their descriptions. Here, let's focus on the catalog `IX/51` which aggregates GRBs from a lot of missions (Fermi, Swift, Agile, ...)

In [ ]:
tap_services_GRBs["IX/51"].describe(verbose=True)

If we were doind our research now, we'd either add more constraints on the words, or wavebands. And then we'd read all these abstracts to select an interresting catalog. Let's create our TAP connexion to VizieR like we did for SIMBAD.

In [ ]:
tap_vizier = pyvo.dal.tap.TAPService("https://tapvizier.cds.unistra.fr/TAPVizieR/tap")

We can find the tables of this catalog by searching the short name from the description in the generic table from `TAP_SCHEMA`

In [ ]:
tap_vizier.search("SELECT * FROM tables where table_name LIKE '%IX/51%'").to_table()

The name of the table is `IX/51/table2`.

It is usually good practice to check the size of a table before downloading it:

In [ ]:
# count allows to count the records before downloading them
count = tap_vizier.search('SELECT count(*) from "IX/51/table2"')
count

`7516` lines is quite reasonable. Let's download it.

In [ ]:
grb_list = tap_vizier.search('SELECT * from "IX/51/table2"').to_table()

# if you have a slower internet connexion, the table can be read from disk too
# grb_list = parse_single_table("grb_list.vot").to_table()

grb_list

In [ ]:
grb_list["astropy_time"] = Time(grb_list["Date"], format="jd").to_datetime()
grb_list.sort("recno")
grb_list

We can also add this to `ipyaladin`

In [ ]:
aladin.add_table(grb_list, color="yellow", shape="circle", source_size=20, name="grb_list")

Now, we'd really like to know wether some of these events have been observed by the XMM space telescope.

## Finding the XMM observation log on VizieR

We can look for the XMM observation log by browing through VizieR's generic `TAP_SCHEMA.tables` and adding a constraint on their descriptions

In [ ]:
tap_vizier.search("""select * from tables where description like '%XMM-Newton%log%'
                  """).to_table()

Let's look at the size of the log:

In [ ]:
tap_vizier.search('select count(*) from "B/xmm/xmmlog"').to_table()

In [ ]:
# retrieve the whole xmmlog table
xmmlog = tap_vizier.search('select * from "B/xmm/xmmlog"').to_table()

# you can uncomment and read from the file if the download time is too long
# xmmlog = parse_single_table("xmmlog.vot").to_table()
xmmlog

We see that some pointings are dumy pointings and have no coordinates. Let's remove them by selecting the data where it's not (`~`) masked

In [ ]:
xmmlog = xmmlog[~xmmlog["RAJ2000"].data.mask]
xmmlog

Let's also remove the observation that did not produce an image (they have "N" in their `Image` column).

In [ ]:
xmmlog = xmmlog[xmmlog["Image"] != "N"]
len(xmmlog)

We can add the center of the observation to ipyaladin.

In [ ]:
aladin.add_table(xmmlog, name="xmmlog_pointings", color="white")

We can already build a MOC from the columns `RAJ2000` and `DEJ2000`. There are the center of the view, and the biggest camera has a field of view of approximatively 0.5°

In [ ]:
xmmlog["lon"] = Longitude(xmmlog["RAJ2000"].data.data, unit="deg")
xmmlog["lat"] = Latitude(xmmlog["DEJ2000"].data.data, unit="deg")
xmmlog

Let's also chose a spatial resolution for the MOC that makes sense compared to the camera's field of view, for example one tenth of it:

In [ ]:
MOC.spatial_resolution_to_order(Angle(0.05 * u.deg))

This is what we will use for the `max_depth` argument

In [ ]:
cones = MOC.from_cones(xmmlog["lon"], xmmlog["lat"], radius=Angle("0.23d"), max_depth=11)
len(cones)

Let's add each of the cones to a new columns

In [ ]:
xmmlog["moc_cones"] = cones

We can also calculate the MOC that is the union of all these cones. This will remove any possible overlaps

In [ ]:
sum_cones = sum(cones)

In [ ]:
aladin.add_moc(sum_cones, fill=True, name="xmm_smoc", alpha=0.2, color="teal", edge=True)

This MOC highlights the fact that some images are missing from the XMM hips. It is updated less often than the observation log. Also note that here we used a very approximated MOC. We could have done a better job by retrieving which camera was used for each pointing and using `MOC.from_polygon` or `MOC.from_astropy_region` for each of them!

However, we can already get some fun information from this. For example, the area covered by XMM from 2001 to today is approximatively: 

In [ ]:
sum_cones.sky_fraction

3% of the sky only! 

We can also check which of the GRBs in our catalog fall within the observations.

![moc contains](moc_contains.png)

**Figure:** The contains methods is extremely useful.

The `contains_skycoords` method allows to check wether some coordinates are within a MOC. It returns a mask which has a value `True` where the point in in the MOC:

In [ ]:
mask = sum_cones.contains_skycoords(SkyCoord(grb_list["RAJ2000"], grb_list["DEJ2000"], frame="icrs"))
mask

In [ ]:
grb_inside = grb_list[mask]
grb_inside

324 events are in the MOC! Let's add them to ipyaladin

In [ ]:
aladin.add_table(grb_inside, shape="triangle", color="red", source_size=20)

## Adding time into the mix

But... Have any of these 324 event been observed by XMM just at the right time? We can also check that programmatically.

To do that, we will use an other flavor of MOCs: the Space-TimeMOCs. This is an other standard of the VO in which allow to save information about when a patch of the sky was observed.

We already added the `astropy_time` to the grb list in anticipation. Let's do the same for the XMM log. First, let's inspect the columns.


Space-Time MOCs (or STMOCs) allow to identify the common space-time regions between our catalog of GRBs and XMM-Newton observations. These come in handy specially for identifing simultaneous observations, which are crucial for multi-wavelength studies of transient events like Gamma-Ray Bursts (GRBs).

This can allow identification by different telescopes, therefore we can know for certain that the identified GRBs are genuine. 

In [ ]:
for column in xmmlog.colnames:
    print(column, ":", xmmlog[column].description)

The times `Obs0` and `Obs1` are the start and end of the observation. Exactly what we need. The `UT` time is expressed in seconds since 2000. This is not a predefined time convention in astropy, so we'll need to define it. Aditionnaly, XMM publishes time in [terestrial time TT](https://xmm-tools.cosmos.esa.int/external/xmm_user_support/documentation/uhb/reftime.html) scale :

In [ ]:
# we have to define a specific time
from astropy.time.formats import TimeFromEpoch
class TimeXMM(TimeFromEpoch):
    name = "time_xmm"
    unit = 1.0 / 86400  # in days (1 day == 86400 seconds)
    epoch_val = "2000-01-01 00:00:00"
    epoch_val2 = None
    epoch_scale = "utc"
    epoch_format = "iso"

We now have everything needed to make an `astropy_time` in `xmmlog` too:

In [ ]:
xmmlog["start_time"] = Time(xmmlog["Obs0"], format="time_xmm", scale="tt")
xmmlog["end_time"] = Time(xmmlog["Obs1"], format="time_xmm", scale="tt")

we can check the start and end of the obseravtion times

In [ ]:
xmmlog["start_time"].sort().iso

We have everything we need to build a Space-Time MOC now. Let's chose its resolution. The TimeMOCs also have helper methods that allow to chose the time resolution:

In [ ]:
TimeMOC.time_resolution_to_order(TimeDelta(0.5, format="jd")) # 0.5 days is order 26

We use the method `STMOC.from_spatial_coverages` that assigns a starting time and an end time to each of the cones that we calculated as approximated field of views.

In [ ]:
stmoc_xmm = STMOC.from_spatial_coverages(
    times_start=xmmlog["start_time"],
    times_end=xmmlog["end_time"],
    spatial_coverages=xmmlog["moc_cones"],
    time_depth=26,
)

In [ ]:
stmoc_xmm.min_time.iso

In [ ]:
stmoc_xmm.max_time.iso

For the GRB list catalog, we don't have an end time, and also since XMM observes at a lower energy, the afterglow could happen up to a few days after the Gamma emmision was measured. This time, since the sources are punctual, let's use `STMOC.from_time_ranges_positions` which assigns a time range to a position:

In [ ]:
stmoc_grb_list = STMOC.from_time_ranges_positions(
    times_start = Time(grb_list["Date"], format="jd"),
    times_end = Time(grb_list["Date"] + 5, format="jd"), # Between immediately after detection and 5 days after
    lon = Longitude(grb_list["RAJ2000"]),
    lat = Latitude(grb_list["DEJ2000"]),
    time_depth = 26, spatial_depth=11
)

In [ ]:
stmoc_intersection = stmoc_grb_list.intersection(stmoc_xmm)

In [ ]:
stmoc_intersection.empty()

The intersection is not empty!

Just for the sake of vizualisation, let's extract the space part of the STMOC to add it to ipyaladin. Ipyaladin does not show STMOC yet, but this will be added in the next versions.

In [ ]:
TimeMOC.n_cells(0)

In [ ]:
space_part_of_intersection = stmoc_intersection.query_by_time(TimeMOC.from_str("0/0-1"))

In [ ]:
aladin.add_moc(space_part_of_intersection, fill=True, name="space_part_of_stmoc_intersection", alpha=0.2, color="green", edge=True)

In [ ]:
candidates_for_cross_identification = grb_list[space_part_of_intersection.contains_lonlat(
    lon = Longitude(grb_list["RAJ2000"]),
    lat = Latitude(grb_list["DEJ2000"])
)]
candidates_for_cross_identification

In [ ]:
aladin.add_table(candidates_for_cross_identification, color="orange",  source_size=20)

And voilà! We got a few sources that fall within the space-time intersection!!! This process can be automatized for different surveys and wavelengths, as long as the observation logs are publicly available.

In [ ]:
# Let's look at the list of simbad results to see if any are in the candidates.
n = 10

aladin.target = SkyCoord(grbs_simbad[n]["ra"], grbs_simbad[n]["dec"], unit="deg")

By now we have seen the power of using STMOCs to identify transients events such as GRBs. This method can also be applied for different short-lived or transient astrophysical objects such as supernova remnants which emit at different wavelengths, motion of bodies from within our solar system, gravitational events... STMOCs is a powerful tool which facilitates the identification of temporal and spatial overlaps, enabling comprehensive multi-wavelength analyses and efficient data management, ultimately advancing our understanding of the objects of interest.